In [2]:
#coding=utf-8
import numpy as np
import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras.optimizers import SGD
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

K.set_image_dim_ordering('tf')

import random
import cv2
import numpy as np

import time

D:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
# READ DATA
path = "G:/fire/python代码/data/td_type/"

# 先把图片存为numpy格式 batchsize * height * weight * channel

blue = np.load(path+"blue.npy")
print(blue.shape)
yellow = np.load(path+"yellow.npy")
print(yellow.shape)
green = np.load(path+"green.npy")
print(green.shape)
white = np.load(path+"white.npy")
print(white.shape)
black = np.load(path+"black.npy")
print(black.shape)
bg = np.load(path+"bg.npy")
print(bg.shape)
#concat
x = np.concatenate((blue,yellow,green,white,black,bg),axis = 0)
print(x.shape)

#["蓝牌","单层黄牌","新能源车牌","白色","黑色-港澳",'bg']
y = np.array([0] * 27370 + [1] * 20828 + [2] * 24032 + [3] * 20667 + [4] * 20180 + [5] * 21815) 
print(len(y),y[:3])

(27370, 9, 34, 3)
(20828, 9, 34, 3)
(24032, 9, 34, 3)
(20667, 9, 34, 3)
(20180, 9, 34, 3)
(21815, 9, 34, 3)
(134892, 9, 34, 3)
134892 [0 0 0]


In [7]:
# 合并 - 随机采样序号 - shuffle 
new_index = list(range(134892 ))
random.shuffle(new_index)
new_x = x[new_index]
new_y = y[new_index]
print(new_x.shape)
print(len(new_y),new_y[:10])

x_train = new_x[:120000]
y_train = new_y[:120000]

x_test = new_x[120000:]
y_test = new_y[120000:]

(134892, 9, 34, 3)
134892 [0 2 0 2 3 2 5 2 5 1]


In [8]:

# 把数值类标签转为10维向量，标签对应位为1其余为0
y_train = keras.utils.np_utils.to_categorical(y_train,6)
y_test = keras.utils.np_utils.to_categorical(y_test, 6)
print(y_train.shape)
print(y_train[0],y_test[0])

#把数据变成float32更精确
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

(120000, 6)
[1. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 1.]
x_train shape: (120000, 9, 34, 3)
120000 train samples
14892 test samples


In [9]:
plateType  = ["蓝牌","单层黄牌","新能源车牌","白色","黑色-港澳","bg"]
def Getmodel_tensorflow(nb_classes):
    # nb_classes = len(charset)

    img_rows, img_cols = 9, 34
    # number of convolutional filters to use
    nb_filters = 32
    # size of pooling area for max pooling
    nb_pool = 2
    # convolution kernel size
    nb_conv = 3

    # x = np.load('x.npy')
    # y = np_utils.to_categorical(range(3062)*45*5*2, nb_classes)
    # weight = ((type_class - np.arange(type_class)) / type_class + 1) ** 3
    # weight = dict(zip(range(3063), weight / weight.mean()))  # 调整权重，高频字优先

    model = Sequential()
    model.add(Conv2D(16, (5, 5),input_shape=(img_rows, img_cols,3)))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(nb_pool, nb_pool)))
    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [10]:
model = Getmodel_tensorflow(6)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 5, 30, 16)         1216      
_________________________________________________________________
activation_1 (Activation)    (None, 5, 30, 16)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 15, 16)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 480)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                30784     
_________________________________________________________________
activation_2 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
__________

In [11]:
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model_checkpoint = ModelCheckpoint(filepath='checkpoint/epoch-{epoch:02d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=False,
                                   mode='auto',
                                   period=1)

callbacks = [model_checkpoint]

model.fit(x_train, y_train, batch_size=128, epochs=60,callbacks=callbacks,
         verbose=1, validation_data=(x_test, y_test))
# score = model.evaluate(x_test, y_test, batch_size=32,verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

Train on 120000 samples, validate on 14892 samples
Epoch 1/60
 52224/120000 [============>.................] - ETA: 14s - loss: 0.6664 - acc: 0.7913

KeyboardInterrupt: 

In [13]:
#test
from keras.models import load_model
model =  load_model('H:/checkpoint/epoch-58_loss-0.0422_val_loss-0.0454.h5')
img = cv2.imdecode(np.fromfile("1.jpg",dtype = np.uint8),-1)
img = cv2.resize(img,(34,9))
d = np.array([img])
print(d.shape)
t = time.time()
pre = model.predict(d)
print(time.time() -t)
tag = ["蓝牌","单层黄牌","新能源车牌","白色","黑色-港澳",'bg']
print("predict: ",tag[np.argmax(pre)])

(1, 9, 34, 3)
0.037836551666259766
predict:  蓝牌
